# OLS MCP Server - Deployment Tests

Tests all 7 tools on the deployed Databricks app at  
`https://ols-mcp-server-3464092709171785.aws.databricksapps.com/mcp`

In [0]:
%pip install --quiet --upgrade httpx databricks-sdk>=0.67 typing_extensions databricks-mcp 
dbutils.library.restartPython()  # Restarting Python environment to apply package upgrades, if necessary, should be done with caution and possibly outside of the current script.

In [0]:
MCP_URL = "https://ols-mcp-server-3464092709171785.aws.databricksapps.com/mcp"

In [0]:
from databricks_mcp import DatabricksMCPClient
from databricks.sdk import WorkspaceClient

# OAuth authentication is required for Databricks Apps
# You need to create a service principal and provide credentials
w = WorkspaceClient(
    host="https://fe-vm-vdm-classic-q6xfh3.cloud.databricks.com",  # Replace with your actual workspace URL
    client_id="",  # Replace with your service principal client ID
    client_secret="",  # Replace with your service principal client secret
    debug_headers=True
)

In [0]:
import json
import httpx
from databricks.sdk import WorkspaceClient


client = httpx.Client(timeout=30.0)

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json, text/event-stream",
}


def mcp_request(method: str, params: dict | None = None, request_id: int = 1) -> dict:
    """Send a JSON-RPC request to the MCP server and return the parsed result."""
    headers = {**HEADERS, **token()}
    if hasattr(mcp_request, "session_id"):
        headers["Mcp-Session-Id"] = mcp_request.session_id

    payload = {"jsonrpc": "2.0", "id": request_id, "method": method}
    if params:
        payload["params"] = params

    resp = w.api_client.do("POST", MCP_URL, headers=headers, json=payload)
    resp.raise_for_status()

    # Capture session ID from first response
    if "mcp-session-id" in resp.headers:
        mcp_request.session_id = resp.headers["mcp-session-id"]

    # Parse SSE response
    for line in resp.text.splitlines():
        if line.startswith("data: "):
            return json.loads(line[6:])
    return {}


def call_tool(name: str, arguments: dict | None = None) -> dict:
    """Call an MCP tool and return the result."""
    result = mcp_request("tools/call", {"name": name, "arguments": arguments or {}})
    return result.get("result", result)


def print_result(data: dict):
    """Pretty-print tool result content."""
    for item in data.get("content", []):
        if item["type"] == "text":
            try:
                print(json.dumps(json.loads(item["text"]), indent=2))
            except json.JSONDecodeError:
                print(item["text"])


print("Client ready.")

## Initialize MCP Session

In [0]:
# Direct REST API call (no MCP protocol wrapper)
app_url = "https://ols-mcp-server-3464092709171785.aws.databricksapps.com/"

# Test health endpoint
response = w.api_client.do("GET", f"{app_url}")
print(f"Health check: {response.json()}")

In [0]:
# List ontologies
response = w.api_client.do("GET", f"{app_url}/mcp/ontologies", timeout=10)
print(f"\nOntologies response status: {response.status_code}")
print(f"First 500 chars: {response.text[:500]}...")

## List Tools

In [0]:
result = mcp_request("tools/list")
tools = result["result"]["tools"]
print(f"Found {len(tools)} tools:\n")
for t in tools:
    params = ", ".join(t["inputSchema"].get("required", []))
    print(f"  - {t['name']}({params})")
    print(f"    {t['description'][:80]}")

## 1. search_terms

In [0]:
result = call_tool("search_terms", {"query": "diabetes", "rows": 5})
print_result(result)

In [0]:
# With ontology filter and exact match
result = call_tool("search_terms", {"query": "asthma", "ontology": "efo", "exact_match": True, "rows": 3})
print_result(result)

## 2. get_ontology_info

In [0]:
result = call_tool("get_ontology_info", {"ontology_id": "efo"})
print_result(result)

## 3. search_ontologies

In [0]:
result = call_tool("search_ontologies", {"search": "disease", "size": 5})
print_result(result)

## 4. get_term_info

In [0]:
# By OBO ID
result = call_tool("get_term_info", {"id": "DUO:0000017"})
print_result(result)

In [0]:
# By IRI
result = call_tool("get_term_info", {"id": "http://purl.obolibrary.org/obo/DOID_9351"})
print_result(result)

## 5. get_term_children

In [0]:
result = call_tool("get_term_children", {
    "term_iri": "http://purl.obolibrary.org/obo/DOID_9351",
    "ontology": "doid",
    "size": 5,
})
print_result(result)

## 6. get_term_ancestors

In [0]:
result = call_tool("get_term_ancestors", {
    "term_iri": "http://purl.obolibrary.org/obo/DOID_9351",
    "ontology": "doid",
    "size": 5,
})
print_result(result)

## 7. find_similar_terms

In [0]:
result = call_tool("find_similar_terms", {
    "term_iri": "http://purl.obolibrary.org/obo/DOID_9351",
    "ontology": "doid",
    "size": 5,
})
print_result(result)